In [1]:
import pickle

graph = pickle.load(file=open('graph.pickle', 'rb'))
index = pickle.load(file=open('index.pickle', 'rb'))

In [ ]:
import numpy as np
from tqdm import tqdm

def create_matrix(clean_graph, index):
    matrix = np.zeros((len(clean_graph), len(clean_graph)))
    for word in tqdm(clean_graph.keys()):
        for wd in clean_graph[word].keys():
            matrix[index[word], index[wd]] = clean_graph[word][wd]
    return matrix

matrix = create_matrix(graph, index)

In [ ]:
from sklearn.decomposition import MiniBatchSparsePCA

pca = MiniBatchSparsePCA(n_components=100, batch_size=1000, normalize_components=True)
pca.fit(matrix)

In [ ]:
import json

with open('graph.json', 'w') as outfile:
    json.dump(graph, outfile)

In [ ]:
with open('graph.json', 'r') as file:
    a = json.load(file)

In [2]:
from scipy.sparse import csr_matrix
from tqdm import tqdm

def create_sparse_matrix(clean_graph, index):
    data = []
    row = []
    col = []
    for word in tqdm(clean_graph.keys()):
        for wd in clean_graph[word].keys():
            row.append(index[word])
            col.append(index[wd])
            data.append(clean_graph[word][wd])
    matrix = csr_matrix((data, (row, col)))
    return matrix
matrix = create_sparse_matrix(graph, index)

100%|██████████| 248804/248804 [00:18<00:00, 13362.22it/s]


In [3]:
n = matrix.shape
n

(248804, 248804)

In [ ]:
from sklearn.decomposition import IncrementalPCA
from tqdm import tqdm
import pickle

chunk_size = 100
n = matrix.shape[0]

pca = IncrementalPCA(n_components=15, batch_size=100)

for i in tqdm(range(0, n//chunk_size)):
    rows = matrix[i*chunk_size : (i+1)*chunk_size].toarray()
    pca.partial_fit(rows)

pickle.dump(file=open('pca.pickle', 'wb'), obj=pca)

In [4]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, random_state=0)

svd.fit(matrix)

TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
       random_state=0, tol=0.0)

In [12]:
transfo_svd = svd.transform(matrix)

In [11]:
import pickle

pickle.dump(file=open('svd.pickle', 'wb'), obj=svd)
pickle.dump(file=open('svd_transfo.pickle', 'wb'), obj=transfo_svd)

In [44]:
import pickle

pca = pickle.load(open("dac/pca.pickle", 'rb'))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator IncrementalPCA from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
transfo = pca.transform(matrix[0:1000].toarray())

In [ ]:
import datetime
a = datetime.datetime.now()
kmeans = KMeans(n_clusters=10, random_state=0)
b = datetime.datetime.now()
print(b-a)

In [ ]:
kmeans.fit(transfo)

In [ ]:
clusters = kmeans.predict(transfo)

In [ ]:
from collections import Counter
a = Counter(clusters)

In [ ]:
import numpy as np
np.zeros(10)

In [ ]:
def create_clusters_index(index, clusters):
    clusters_index = {}
    for word, row in index.values():
        clusters_index[word] = clusters[row]
    return clusters_index

In [ ]:
import numpy as np

def book_to_vect(clusters_index, n_clusters, book):
    """
    kmeans is an index of kmeans[word] = cluster_id
    """
    vect = np.zeros(n_clusters)
    for word in book.author:
        if word in clusters_index.keys():
            vect[clusters_index[word]] += 1
    for word in book.title:
        if word in clusters_index.keys():
            vect[clusters_index[word]] += 1
    vect = np.linalg.norm(vect)
    return vect

In [ ]:
a = [4,5,3,7,6]

In [ ]:
a = np.array(a)

In [ ]:
np.linalg.norm(a)